In [1]:
import os
import logging
from pathlib import Path
from typing import Optional
from openai import OpenAI
from dotenv import load_dotenv

### Meeting Summarization

In [ ]:
client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY', ''))

def build_prompt(meeting_transcript: str) -> str:
    try:
        template = {
            'system_role': "당신은 회의 전체 내용을 카테고리별로 요약해주는 요약 AI입니다.",
            'output_format': [
                "다음 회의 날짜와 시간: OOO",
                "아이템: OOO",
                "해야 할 일: OOO",
                "팀원들의 의견: OOO",
                "멘토의 피드백: OOO"
            ]
        }

        prompt = f""" {template['system_role']}
아래 회의 전체 내용을 다음 형식으로 요약하세요:

[출력 예시]
{chr(10).join(template['output_format'])}

[회의 전체 내용]
{meeting_transcript}

[정리된 결과] """
        return prompt

    except Exception as e:
        logging.error(f"Failed to build prompt: {str(e)}")
        raise

def summarize_meeting(transcript: str):
    prompt = build_prompt(transcript)


    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "당신은 회의 전체 내용을 카테고리별로 요약해주는 요약 AI입니다."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,
    )


    return response.choices[0].message.content.strip()

# 테스트용
meeting_transcript = """
회의 시작하겠습니다. 모두 접속한 것 같네요. 오늘은 API 리스크 보험 모델 설계와 관련된 내용을 중점적으로 다뤄볼게요. 지난주에 각자 맡은 부분들 어떻게 진행됐는지 공유부터 해볼까요?

데이터 수집부터 얘기할게요. 공공 데이터 API는 대부분 연결했는데, SLA 위반 기준이 정확히 명시된 건 많지 않아요. 특히 중소 규모 API는 관련 데이터가 누락된 게 많더라고요. 장애 기록도 일부는 1년 이상 안 올라온 것도 있었고요.

그럼 그 기준은 어떻게 정리하고 있어요? 가이드라인 같은 거 임의로 세워봤어요?

네. 일단 일일 호출량 기준 1천 건 이상인 API를 '주요 API'로 분류하고, 여기에 장애 빈도랑 지속 시간을 결합해서 등급을 나눴어요. 근데 아직 통계적으로 유의미한 결과는 못 냈고요.

오케이. 그럼 보험 설계 쪽은요?

저는 보험료 책정 모델 초안을 만들어봤어요. 호출량, 장애 확률, 지속 시간 3가지를 주요 변수로 넣어서 회귀 분석 돌려봤는데, 아직 잔차가 꽤 크더라고요. 특히 batch API보다 실시간 API에서 예측 오차가 컸어요.

그럴 수 있어요. 실시간 API는 트래픽이 예측 불가능하니까. 그럼 이걸 구간별로 나눠서 분포를 다시 볼 수 있을까요?

가능하죠. 0~1000, 1000~10000, 10000 이상으로 나눠서 다시 돌려볼게요.

좋아요. 그리고 리스크 시나리오는 어떻게 정리되고 있어요?

실제 장애 사례를 기반으로 3가지 케이스 만들었어요. 1번은 호출량 급증, 2번은 외부 API 장애 전이, 3번은 인증 실패 반복이에요. 여기에 대해 보상 조건을 다르게 적용해봤어요.

그거 괜찮네요. 근데 멘토님이 지난번에 시나리오가 너무 일반적이라고 하셨잖아요. 혹시 최근 핀테크 쪽 이슈 기반으로 리스크 요소 추가할 수 있을까요?

그 부분 제가 찾아볼게요. 실제로 인증 API 장애로 결제 오류난 사건이 몇 건 있더라고요. 사례 정리해서 공유드릴게요.

좋아요. 그럼 다음 회의는 다음 주 화요일 오전 10시로 정할게요. 그때까지 각자 정리된 모델이랑 시나리오 안 가져오시면 회의 길어질 수 있어요!

네 알겠습니다. 정리해서 올리겠습니다.

수고하셨습니다!
"""

if __name__ == "__main__":

    if not os.environ.get('OPENAI_API_KEY'):
        api_key = input("OpenAI API 키를 입력하세요: ")
        os.environ['OPENAI_API_KEY'] = api_key
        client = OpenAI(api_key=api_key)


    try:
        summary = summarize_meeting(meeting_transcript)
        print(summary)
    except Exception as e:
        logging.error(f"요약 중 오류 발생: {str(e)}")


2025-05-06 22:59:43,970 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


다음 회의 날짜와 시간: 다음 주 화요일 오전 10시  
아이템: API 리스크 보험 모델 설계  
해야 할 일: 각자 정리된 모델과 시나리오를 다음 회의까지 준비하기  
팀원들의 의견:  
- 데이터 수집: 공공 데이터 API 연결 완료, SLA 위반 기준 명시 부족  
- 보험료 책정 모델 초안: 호출량, 장애 확률, 지속 시간을 변수로 사용, 예측 오차 존재  
- 리스크 시나리오: 3가지 케이스(호출량 급증, 외부 API 장애 전이, 인증 실패 반복) 설정, 멘토 피드백 반영 필요  
멘토의 피드백: 시나리오가 일반적이라는 지적, 최근 핀테크 이슈 기반 리스크 요소 추가 필요


### Task Alignment

In [ ]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

TEAM_TRAITS = """
혜주: 직감형, 창의성 있음, 활동성 중간, 성과 지향적 리더십(카리스마적 리더십에 가까움), 신중한 관계 성향, 실행력 중간 
현서: 직감형, 현실 지향형, 활동성 중간, 성과 지향적(카리스마에 가까움), 관계에 대해 많이 신중, 실행력 중간 이하 
윤조: 분석형, 창의력 높음, 활동성 중간, 안정적 리더십형, 관계 독립형, 실행력 중간 
정아: 직감형, 현실 지향형, 활동성 중간, 안정형 리더십형, 내향형, 책임감 높음, 실행력 높음 
윤희: 현실적인 편, 정적 활동형, 부드러운 리더, 헌신적 관계형, 실행력 중상 
"""

TASKS = """
1. API 연결 상태와 SLA 기준 정리
2. 보험료 책정 모델 개선 (회귀 분석 정교화)
3. 실시간 API 예측 오차 개선을 위한 데이터 구간화 분석
4. 핀테크 이슈 기반 리스크 시나리오 보완
5. 시나리오 정리본 작성 및 발표 준비
"""

def build_task_prompt(tasks: str, team_traits: str) -> str:
    return f"""
당신은 팀원 성향을 고려해 업무를 적절히 분배하는 AI입니다.

[팀원 성향]
{team_traits}

[해야 할 일 목록]
{tasks}

- 각 업무에 대해 어떤 팀원이 적합한지 1명씩 지정하고, 그 이유도 간단히 설명하세요.
- 설명에 포함되는 역량은 해당 인물의 실제 성향과 일치해야 합니다.
- 예: 창의성이 높지 않은 인물에게는 창의성과 관련된 설명을 하지 마세요.
- 성향의 정도를 잘 구분해주세요. (예: '중간', '있음', '높음' 등 정도를 나타내는 표현 구분)

[출력 형식 예시]
1. 업무명 - 담당자: 이름 (이유: 성향과 업무 특성의 연결)
"""

def assign_tasks(tasks: str, team_traits: str) -> str:
    prompt = build_task_prompt(tasks, team_traits)

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )

    return response.choices[0].message.content.strip()

if __name__ == "__main__":
    try:
        result = assign_tasks(TASKS, TEAM_TRAITS)
        print("업무 배정 결과:\n")
        print(result)
    except Exception as e:
        logging.error(f"업무 배정 실패: {str(e)}")


2025-05-06 23:19:08,073 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


업무 배정 결과:

1. API 연결 상태와 SLA 기준 정리 - 담당자: 윤조 (이유: 분석형으로서 데이터 분석 능력이 뛰어나고, 안정적 리더십형으로 팀 내에서 신뢰를 줄 수 있어 체계적인 정리가 가능함)

2. 보험료 책정 모델 개선 (회귀 분석 정교화) - 담당자: 정아 (이유: 책임감이 높고 실행력이 높아 회귀 분석과 같은 정교한 작업을 체계적으로 수행할 수 있으며, 현실 지향형으로 실질적인 결과를 도출할 수 있음)

3. 실시간 API 예측 오차 개선을 위한 데이터 구간화 분석 - 담당자: 혜주 (이유: 창의성이 있어 데이터 구간화에 대한 새로운 접근 방식을 제안할 수 있으며, 직감형으로 빠른 판단이 가능함)

4. 핀테크 이슈 기반 리스크 시나리오 보완 - 담당자: 현서 (이유: 현실 지향형으로 핀테크 이슈를 분석하고, 신중한 관계 성향으로 다양한 의견을 수렴하여 리스크 시나리오를 보완할 수 있음)

5. 시나리오 정리본 작성 및 발표 준비 - 담당자: 윤희 (이유: 부드러운 리더십과 헌신적인 관계형으로 팀원들과의 협업을 통해 시나리오를 잘 정리하고 발표 준비를 할 수 있음)
